In [126]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_before)]
    after_season =  leagues_value[(leagues_value['PlayerID'] == player_id) & (leagues_value['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * .8), "Only Before"
                else:
                    return (float(add_correct_money(before_season.iloc[0]['Market Value'])) * 1.2), "Only Before"
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * .8), "Only After"
                else:
                    return (float(add_correct_money(after_season.iloc[0]['Market Value'])) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        #print('hi')
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        #print(before_season_val, after_season_val)
        if before_season_val != "-" and after_season_val != "-":
            # if type(before_season_val) == str:
            #     before_season_val = add_correct_money(before_season_val)
            # if type(after_season_val) == str:
            #     after_season_val = add_correct_money(after_season_val)
            return ((float(add_correct_money(before_season.iloc[0]['Market Value'])) + float(add_correct_money(after_season.iloc[0]['Market Value'])) / 2)), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * .8)), "Used Season Before * .8 But Had Both"
            else:
                return (float(add_correct_money(before_season.iloc[0]['Market Value']) * 1.2)), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * .8)), "Used Season After * .8 But Had Both"
            else:
                return (float(add_correct_money(after_season.iloc[0]['Market Value']) * 1.2)), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, database_name):
    #print(player_name, country_code, season)
    row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == season) & (database_name['Team 1 Code'] == country_code)]
    one_up_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)+1) & (database_name['Team 1 Code'] == country_code)]
    one_down_row = database_name[(database_name['Name'] == player_name) & (database_name['Season'] == int(season)-1) & (database_name['Team 1 Code'] == country_code)]
    #print(len(row), len(one_down_row), len(one_up_row))
    final_value = 0
    if not row.empty:
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if type(marketval) == str:
            marketval = float(add_correct_money(marketval))
        if marketval != 0:
            #print('***', player_name, marketval)
            return marketval, "Found in Curr Season"
        else:
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"

def last_name_match(name, dataset_of_names):
    matching_names = []
    for full_name in dataset_of_names:
        
        # Split the full name into tokens
        full_name_tokens = set(re.split(r'\s+|[-–]', full_name))
        
        # Check if any token matches the last name or its unidecoded version
        for token in full_name_tokens:
            #if full_name == 'Fródi Benjaminsen':
                #print(token, name, token == name)
                #print(full_name_tokens)
            
            if token == name or unidecode(token) == name:
                matching_names.append(full_name)
                break  # Once a match is found, move to the next name
    return matching_names

def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    #print(game_row)
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    #print(original_country_code)
    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    #print(f"1{team1_code}, 2{team2_code}")
    if original_country_code == 'CG':
        original_country_code = ' CG'
    elif original_country_code  == 'NI':
        original_country_code = ' NI'
    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    
    return possible_names[0]

def add_correct_money(money_column):
    if money_column == 0:
        return 0

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0

def remove_nation(match, nation1):
    # Split the match string by 'vs' and strip any leading or trailing spaces
    parts = [part.strip() for part in match.split('vs')]
    # Remove the nation1 from the list of parts
    parts.remove(nation1)
    # Join the remaining parts and return
    return ''.join(parts)


In [2]:
#loading CSVs

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_57573/3266352280.py:7: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [3]:
combined_df = load_csv_dataset('combined_df_may_1.csv')
combined_df = combined_df.drop(columns={'Unnamed: 0'})

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_57573/2739104453.py:49: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


## Weird 0 Cases

In [14]:
pink_group_0_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - odd_name_found_0_cases.csv')
pink_group_0_cases = pink_group_0_cases.drop(columns={'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'To Remove'})

In [53]:
for index, row in pink_group_0_cases.iterrows():
    if row['Nationality'] == 'Namibia':
        pink_group_0_cases.at[index, 'Team Country Code'] = 'NB'
    elif row['Nationality'] == 'Congo':
        pink_group_0_cases.at[index, 'Nationality'] = 'Republic of the Congo'
    elif row['Nationality'] == 'Republic of the Congo':
        pink_group_0_cases.at[index, 'Team Country Code'] = 'CG'

    

In [92]:
pink_group_0_cases['Status New'] = ''

pink_group_0_cases['Market Value New'] = 0

In [73]:
def vet_group_using_initials(jersey, names_found, names_same_nation_last_name):
    correct_names_list = []

    jersey_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', jersey)]

    names_found_tokens_initials = [token[0] for token in re.split(r'\s+|[-–]', names_found)]

    for name in names_same_nation_last_name:
        name_initials = [token[0] for token in re.split(r'\s+|[-–]', name)]
        
        if set(jersey_tokens_initials) == set(name_initials):
            correct_names_list.append(name)
        elif set(jersey_tokens_initials).issubset(set(name_initials)):
            correct_names_list.append(name)
        #else:
            #print('fail')

    if set(jersey_tokens_initials) == set(names_found_tokens_initials):
        correct_names_list.append(names_found)
    elif set(jersey_tokens_initials).issubset(set(names_found_tokens_initials)):
            correct_names_list.append(names_found)


    return correct_names_list

In [133]:
success_counter = 0
done_counter = 0
for index, row in pink_group_0_cases.iterrows():
    
    if row['Status New'] != 'Done':
        correct_name = ''
        player_this = row['ORIGINAL JERSEY']
        if player_this == 'Král':
            player_this = 'Alex Kral'
        elif player_this == 'Oršić':
            player_this = 'Mislav Orsic'
        elif player_this == 'Višća':
            player_this = 'Edin Visca'
        elif player_this == 'evčík':
            player_this = 'Petr Sevcik'
        nationality = row['Nationality']
        date = row['Date']
        match = row['Match']
        
        full_num = row['Season']
        
        if nationality == 'Republic of the Congo' or nationality == 'Namibia':
            country_code_this = row['Team Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            
            best_match = threshold_player_match(player_this, dataset_nationality)[0]
            
            if len(best_match) == 1:
                correct_name = best_match[0]
                print(index, nationality, player_this)
                #print(index, best_match)
            else:
                print(index, nationality, player_this)
                #print(index, best_match)

                last_name = re.split(r'\s+|[-–]', player_this)[-1]
            # #TRY TO FIND OTHER PLAYERS FROM THE NATION W THE SAME LAST NAME
                names_same_nation_last_name = last_name_match(last_name, dataset_nationality)

                vetted_list_using_initials = vet_group_using_initials(player_this, player_this, best_match)
                if player_this in vetted_list_using_initials:
                    vetted_list_using_initials.remove(player_this)
                #print(index, vetted_list_using_initials)
                correct_name = vetted_list_using_initials[0]
            

            number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
            #print(player_this, reason, add_correct_money(number))

            #add a new row to lookup dataset 1 
            if type(number) == float:
                0==0
            else:
                number = add_correct_money(number)
            print('**')
            print(index, player_this, correct_name, number, reason)
            if reason == 'Found in Curr Season':
                pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                pink_group_0_cases.at[index, 'Status New'] = 'Done'
                pink_group_0_cases.at[index, 'Market Value New'] = number
                pink_group_0_cases.at[index, 'Match Case'] = 'single'
                success_counter += 1
                
        else:
            country_code_this = row['Team Country Code']
            dataset_nationality = leagues_value[(leagues_value['Team 1 Code'] == country_code_this) & 
                                        (leagues_value['Season'].isin([full_num, full_num + 1, full_num - 1]))]['Name'].unique()
            
            best_match = threshold_player_match(player_this, dataset_nationality)[0]
            
            if len(best_match) >= 2:
                print(index, best_match)
                vetted_list_using_initials = vet_group_using_initials(player_this, player_this, best_match)
                if player_this in vetted_list_using_initials:
                    vetted_list_using_initials.remove(player_this)
                
                for name in vetted_list_using_initials:
                    if player_this in unidecode(name):
                        0==0
                    elif unidecode(player_this) in name:
                        0==0
                    else:
                        vetted_list_using_initials.remove(name)
                if len(vetted_list_using_initials) == 1:
                    correct_name = vetted_list_using_initials[0]
                #print(index, nationality, player_this, vetted_list_using_initials)

                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                #print(player_this, reason, add_correct_money(number))

                #add a new row to lookup dataset 1 
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)
                print('**')
                print(index, player_this, correct_name, number, reason)
                if reason == 'Found in Curr Season':
                    pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_0_cases.at[index, 'Status New'] = 'Done'
                    pink_group_0_cases.at[index, 'Market Value New'] = number
                    pink_group_0_cases.at[index, 'Match Case'] = 'single'
                    success_counter += 1
                
            else:
                #print(index, player_this, best_match)
                correct_name = best_match[0]
                number, reason = find_in_transfermarkt(correct_name, full_num, country_code_this, leagues_value)
                #success_counter += 1
                if type(number) == float:
                    0==0
                else:
                    number = add_correct_money(number)
                print('**')
                print(index, player_this, correct_name, number, reason)
                if reason == 'Found in Curr Season':
                    pink_group_0_cases.at[index, 'Name(s) Found'] = correct_name
                    pink_group_0_cases.at[index, 'Status New'] = 'Done'
                    pink_group_0_cases.at[index, 'Market Value New'] = number
                    pink_group_0_cases.at[index, 'Match Case'] = 'single'
                    success_counter += 1


            


                # if len(vetted_list_using_initials) == 1:
                #     correct_name = vetted_list_using_initials[0]
                # elif len(vetted_list_using_initials) >= 2:
                #     initial_double_letters = multiple_occurrences(jersey_tokens_initials)
                #     vetted_list_using_initials = vet_using_repeated_letters(initial_double_letters, vetted_list_using_initials)
    else:
        done_counter += 1
print(success_counter, done_counter)

**
155 Petr Sevcik Petr Sevcik 1600000.0 Found in Curr Season
1 719


In [99]:
for i in range(0, len(pink_group_0_cases)):
    if pink_group_0_cases.at[i, 'Market Value New'] != 0:
        if pink_group_0_cases.at[i, 'Market Value New'] != pink_group_0_cases.at[i, 'Market Value']:
            print(i)
        #print(pink_group_0_cases.at[i, 'Market Value New'], pink_group_0_cases.at[i, 'Market Value'])

In [134]:
pink_group_0_cases[pink_group_0_cases['Status New'] == 'Done']['Lookup Return Case'].value_counts()

Lookup Return Case
working    720
Name: count, dtype: int64

In [142]:
for i in range(0, len(pink_group_0_cases)):
    if pink_group_0_cases.at[i, 'Market Value'] == pink_group_0_cases.at[i, 'Market Value New']:
        0==0
    else:
        print(i)

In [144]:
pink_group_0_cases_final = pink_group_0_cases.drop(columns={'Status New', 'Market Value New'})
pink_group_0_cases_final.to_csv('pink_group_weird_0_cases.csv')

## Online Lookup - 0 Names Found one

In [148]:
pink_group_lookup_cases = load_csv_dataset('April 25 - CONCACAF & CONMEBOL OUT - DATA CSVs/Pink Group/CONCACAF & CONMEBOL OUT - Online lookup required - 0 names found.csv')

In [ ]:
#separate into 

#olympics and U20 world cup

#namibia folks 


#check out the martinique / guadeloupe rows in TRANSFERMARKT data


#if the season is 2013 search the old dataset 